In [1]:
import argparse
import numpy as np
import shlex
import subprocess
import sys
import wave


In [2]:
from deepspeech import Model, printVersions
from timeit import default_timer as timer

In [3]:
try:
    from shhlex import quote
except ImportError:
    from pipes import quote


In [4]:
# Define the sample rate for audio

SAMPLE_RATE = 16000
# These constants control the beam search decoder

# Beam width used in the CTC decoder when building candidate transcriptions
BEAM_WIDTH = 500

# The alpha hyperparameter of the CTC decoder. Language Model weight
LM_ALPHA = 0.75

# The beta hyperparameter of the CTC decoder. Word insertion bonus.
LM_BETA = 1.85


# These constants are tied to the shape of the graph used (changing them changes
# the geometry of the first layer), so make sure you use the same constants that
# were used during training

# Number of MFCC features to use
N_FEATURES = 26

# Size of the context window used for producing timesteps in the input vector
N_CONTEXT = 9


In [5]:
def convert_samplerate(audio_path):
    sox_cmd = 'sox {} --type raw --bits 16 --channels 1 --rate {} --encoding signed-integer --endian little --compression 0.0 --no-dither - '.format(quote(audio_path), SAMPLE_RATE)
    try:
        output = subprocess.check_output(shlex.split(sox_cmd), stderr=subprocess.PIPE)
    except subprocess.CalledProcessError as e:
        raise RuntimeError('SoX returned non-zero status: {}'.format(e.stderr))
    except OSError as e:
        raise OSError(e.errno, 'SoX not found, use {}hz files or install it: {}'.format(SAMPLE_RATE, e.strerror))

    return SAMPLE_RATE, np.frombuffer(output, np.int16)


def metadata_to_string(metadata):
    return ''.join(item.character for item in metadata.items)


class VersionAction(argparse.Action):
    def __init__(self, *args, **kwargs):
        super(VersionAction, self).__init__(nargs=0, *args, **kwargs)

    def __call__(self, *args, **kwargs):
        printVersions()
        exit(0)



In [8]:
alphabet = '/home/absin/Downloads/deepspeech-0.5.1-models/alphabet.txt'
model = '/home/absin/Downloads/deepspeech-0.5.1-models/output_graph.pb'
lm = '/home/absin/Downloads/deepspeech-0.5.1-models/lm.binary'
trie = '/home/absin/Downloads/deepspeech-0.5.1-models/trie'

In [9]:
print('Loading model from file {}'.format(model), file=sys.stderr)
model_load_start = timer()
ds = Model(model, N_FEATURES, N_CONTEXT, alphabet, BEAM_WIDTH)
model_load_end = timer() - model_load_start
print('Loaded model in {:.3}s.'.format(model_load_end), file=sys.stderr)


Loading model from file /home/absin/Downloads/deepspeech-0.5.1-models/output_graph.pb
Loaded model in 0.0963s.


In [10]:
print('Loading language model from files {} {}'.format(lm, trie), file=sys.stderr)
lm_load_start = timer()
ds.enableDecoderWithLM(alphabet, lm, trie, LM_ALPHA, LM_BETA)
lm_load_end = timer() - lm_load_start
print('Loaded language model in {:.3}s.'.format(lm_load_end), file=sys.stderr)

Loading language model from files /home/absin/Downloads/deepspeech-0.5.1-models/lm.binary /home/absin/Downloads/deepspeech-0.5.1-models/trie
Loaded language model in 0.159s.


In [24]:
print(dir(ds))
import time

['__class__', '__del__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_impl', 'enableDecoderWithLM', 'feedAudioContent', 'finishStream', 'finishStreamWithMetadata', 'intermediateDecode', 'setupStream', 'stt', 'sttWithMetadata']


In [42]:
from scipy.io import wavfile
fs, frames = wavfile.read('/home/absin/Downloads/RE16255e9fec360777c31418c2853372dc_1.wav')

In [45]:
frames.shape[0]

1518720

In [44]:
frames[0:4000]

array([8, 7, 4, ..., 3, 4, 5], dtype=int16)

In [48]:
stream_context = ds.setupStream()
start = 0
time.time()
while(start<frames.shape[0]):
    ds.feedAudioContent(stream_context, np.frombuffer(frames[start:start+4000], np.int16))
    start += 4000
    print('Done '+str(start/16000))
    ds.intermediateDecode(stream_context)
    time.sleep(0.25)
text = ds.finishStream(stream_context)
print("Recognized: %s" % text)

Done 0.25
Done 0.5
Done 0.75
Done 1.0
Done 1.25
Done 1.5
Done 1.75
Done 2.0
Done 2.25
Done 2.5
Done 2.75
Done 3.0
Done 3.25
Done 3.5
Done 3.75
Done 4.0
Done 4.25
Done 4.5
Done 4.75
Done 5.0
Done 5.25
Done 5.5
Done 5.75
Done 6.0
Done 6.25
Done 6.5
Done 6.75
Done 7.0
Done 7.25
Done 7.5
Done 7.75
Done 8.0
Done 8.25
Done 8.5
Done 8.75
Done 9.0
Done 9.25
Done 9.5
Done 9.75
Done 10.0
Done 10.25
Done 10.5
Done 10.75
Done 11.0
Done 11.25
Done 11.5
Done 11.75
Done 12.0
Done 12.25
Done 12.5
Done 12.75
Done 13.0
Done 13.25
Done 13.5
Done 13.75
Done 14.0
Done 14.25
Done 14.5
Done 14.75
Done 15.0
Done 15.25
Done 15.5
Done 15.75
Done 16.0
Done 16.25
Done 16.5
Done 16.75
Done 17.0
Done 17.25
Done 17.5
Done 17.75
Done 18.0
Done 18.25
Done 18.5
Done 18.75
Done 19.0
Done 19.25
Done 19.5
Done 19.75
Done 20.0
Done 20.25
Done 20.5
Done 20.75
Done 21.0
Done 21.25


KeyboardInterrupt: 